# 📊 Avaliação de Modelos - NeuroTranslator PT-EN

Este notebook implementa métricas de avaliação para os modelos de tradução, incluindo BLEU, ROUGE, METEOR e análises qualitativas.

## Métricas Implementadas:
- **BLEU**: Bilingual Evaluation Understudy
- **ROUGE**: Recall-Oriented Understudy for Gisting Evaluation
- **METEOR**: Metric for Evaluation of Translation with Explicit ORdering
- **ChrF**: Character n-gram F-score
- **Análise Qualitativa**: Exemplos e casos de erro

In [ ]:
# Importações necessárias
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import json
import re
from datetime import datetime

# Métricas de avaliação
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import sacrebleu

# Processamento de texto
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy

# Modelos
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Bibliotecas importadas com sucesso!")

## 1. Carregamento dos Modelos e Dados de Teste

In [ ]:
# Carregar dados de teste
def load_test_data():
    """
    Carrega dados de teste para avaliação
    """
    # Dados de teste sintéticos (substituir por dados reais)
    test_data = {
        'portuguese': [
            "Olá, como você está hoje?",
            "Eu gosto muito de programar em Python.",
            "O tempo está muito bom para sair.",
            "Vamos trabalhar juntos neste projeto interessante.",
            "A inteligência artificial está revolucionando o mundo.",
            "Preciso aprender mais sobre redes neurais profundas.",
            "Este é um sistema de tradução automática avançado.",
            "Vou estudar machine learning durante as férias.",
            "A tecnologia está evoluindo muito rapidamente.",
            "Gosto de resolver problemas complexos de programação."
        ],
        'english_reference': [
            "Hello, how are you today?",
            "I really like programming in Python.",
            "The weather is very good for going out.",
            "Let's work together on this interesting project.",
            "Artificial intelligence is revolutionizing the world.",
            "I need to learn more about deep neural networks.",
            "This is an advanced automatic translation system.",
            "I will study machine learning during the holidays.",
            "Technology is evolving very rapidly.",
            "I like solving complex programming problems."
        ]
    }
    
    return pd.DataFrame(test_data)

# Carregar dados de teste
test_df = load_test_data()
print(f"📊 Dados de teste carregados: {len(test_df)} pares")
test_df.head()

In [ ]:
# Simulador de traduções dos modelos (substituir por modelos reais)
class ModelSimulator:
    """
    Simula traduções de diferentes modelos para demonstração
    """
    def __init__(self, model_name):
        self.model_name = model_name
        
        # Traduções simuladas com diferentes qualidades
        self.translations = {
            "Olá, como você está hoje?": {
                "cnn_rnn": "Hello, how you are today?",
                "transformer": "Hello, how are you today?",
                "t5_finetuned": "Hello, how are you today?"
            },
            "Eu gosto muito de programar em Python.": {
                "cnn_rnn": "I like very much programming in Python.",
                "transformer": "I really like programming in Python.",
                "t5_finetuned": "I really like to program in Python."
            },
            "O tempo está muito bom para sair.": {
                "cnn_rnn": "The time is very good for go out.",
                "transformer": "The weather is very good for going out.",
                "t5_finetuned": "The weather is very nice to go out."
            },
            "Vamos trabalhar juntos neste projeto interessante.": {
                "cnn_rnn": "Let's work together in this interesting project.",
                "transformer": "Let's work together on this interesting project.",
                "t5_finetuned": "Let's work together on this interesting project."
            },
            "A inteligência artificial está revolucionando o mundo.": {
                "cnn_rnn": "The artificial intelligence is revolutionizing world.",
                "transformer": "Artificial intelligence is revolutionizing the world.",
                "t5_finetuned": "Artificial intelligence is revolutionizing the world."
            }
        }
    
    def translate(self, text):
        if text in self.translations:
            return self.translations[text].get(self.model_name, "[Translation not available]")
        else:
            # Tradução genérica para textos não mapeados
            return f"[{self.model_name} translation of: {text}]"

# Criar simuladores dos modelos
models = {
    'cnn_rnn': ModelSimulator('cnn_rnn'),
    'transformer': ModelSimulator('transformer'),
    't5_finetuned': ModelSimulator('t5_finetuned')
}

print("🤖 Simuladores de modelos criados!")

## 2. Implementação das Métricas de Avaliação

In [ ]:
class TranslationEvaluator:
    """
    Classe para avaliar qualidade de traduções
    """
    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1
        
        # Download recursos NLTK se necessário
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')
            
        try:
            nltk.data.find('corpora/wordnet')
        except LookupError:
            nltk.download('wordnet')
    
    def calculate_bleu(self, reference, hypothesis):
        """
        Calcula BLEU score
        """
        reference_tokens = [word_tokenize(reference.lower())]
        hypothesis_tokens = word_tokenize(hypothesis.lower())
        
        # BLEU-1, BLEU-2, BLEU-3, BLEU-4
        bleu_scores = {}
        for n in range(1, 5):
            weights = [1/n] * n + [0] * (4-n)
            bleu_scores[f'bleu_{n}'] = sentence_bleu(
                reference_tokens, hypothesis_tokens, 
                weights=weights, smoothing_function=self.smoothing
            )
        
        return bleu_scores
    
    def calculate_rouge(self, reference, hypothesis):
        """
        Calcula ROUGE scores
        """
        scores = self.rouge_scorer.score(reference, hypothesis)
        
        rouge_scores = {}
        for metric, score in scores.items():
            rouge_scores[f'{metric}_precision'] = score.precision
            rouge_scores[f'{metric}_recall'] = score.recall
            rouge_scores[f'{metric}_f1'] = score.fmeasure
        
        return rouge_scores
    
    def calculate_meteor(self, reference, hypothesis):
        """
        Calcula METEOR score
        """
        try:
            score = meteor_score([word_tokenize(reference)], word_tokenize(hypothesis))
            return {'meteor': score}
        except:
            return {'meteor': 0.0}
    
    def calculate_chrf(self, reference, hypothesis):
        """
        Calcula ChrF score (Character n-gram F-score)
        """
        try:
            score = sacrebleu.sentence_chrf(hypothesis, [reference])
            return {'chrf': score.score / 100.0}  # Normalizar para 0-1
        except:
            # Implementação simplificada se sacrebleu não estiver disponível
            ref_chars = set(reference.lower().replace(' ', ''))
            hyp_chars = set(hypothesis.lower().replace(' ', ''))
            
            if len(hyp_chars) == 0:
                return {'chrf': 0.0}
            
            precision = len(ref_chars & hyp_chars) / len(hyp_chars)
            recall = len(ref_chars & hyp_chars) / len(ref_chars) if len(ref_chars) > 0 else 0
            
            if precision + recall == 0:
                f1 = 0.0
            else:
                f1 = 2 * (precision * recall) / (precision + recall)
            
            return {'chrf': f1}
    
    def evaluate_translation(self, reference, hypothesis):
        """
        Avalia uma tradução com todas as métricas
        """
        metrics = {}
        
        # BLEU scores
        metrics.update(self.calculate_bleu(reference, hypothesis))
        
        # ROUGE scores
        metrics.update(self.calculate_rouge(reference, hypothesis))
        
        # METEOR score
        metrics.update(self.calculate_meteor(reference, hypothesis))
        
        # ChrF score
        metrics.update(self.calculate_chrf(reference, hypothesis))
        
        return metrics

# Criar avaliador
evaluator = TranslationEvaluator()
print("📏 Avaliador de traduções criado!")

## 3. Avaliação dos Modelos

In [ ]:
# Gerar traduções e avaliar
def evaluate_models(test_df, models, evaluator):
    """
    Avalia todos os modelos no conjunto de teste
    """
    results = defaultdict(list)
    detailed_results = []
    
    for idx, row in test_df.iterrows():
        source = row['portuguese']
        reference = row['english_reference']
        
        print(f"\n📝 Avaliando: '{source}'")
        print(f"🎯 Referência: '{reference}'")
        
        for model_name, model in models.items():
            # Gerar tradução
            hypothesis = model.translate(source)
            print(f"🤖 {model_name}: '{hypothesis}'")
            
            # Avaliar tradução
            metrics = evaluator.evaluate_translation(reference, hypothesis)
            
            # Armazenar resultados
            result_entry = {
                'model': model_name,
                'source': source,
                'reference': reference,
                'hypothesis': hypothesis,
                **metrics
            }
            detailed_results.append(result_entry)
            
            # Adicionar às médias
            for metric, value in metrics.items():
                results[f'{model_name}_{metric}'].append(value)
    
    return results, detailed_results

# Executar avaliação
print("🚀 Iniciando avaliação dos modelos...")
results, detailed_results = evaluate_models(test_df[:5], models, evaluator)  # Usar apenas 5 exemplos para demonstração

print("\n✅ Avaliação concluída!")

## 4. Análise dos Resultados

In [ ]:
# Calcular médias das métricas
def calculate_average_metrics(results):
    """
    Calcula médias das métricas por modelo
    """
    avg_metrics = {}
    
    for key, values in results.items():
        if values:  # Verificar se a lista não está vazia
            avg_metrics[key] = np.mean(values)
    
    return avg_metrics

# Calcular médias
avg_metrics = calculate_average_metrics(results)

# Organizar resultados por modelo
model_metrics = defaultdict(dict)
for key, value in avg_metrics.items():
    model_name, metric_name = key.split('_', 1)
    model_metrics[model_name][metric_name] = value

# Criar DataFrame para visualização
metrics_df = pd.DataFrame(model_metrics).T
print("📊 Métricas médias por modelo:")
print(metrics_df.round(4))

In [ ]:
# Visualizar principais métricas
main_metrics = ['bleu_4', 'rouge1_f1', 'rougeL_f1', 'meteor', 'chrf']
available_metrics = [m for m in main_metrics if m in metrics_df.columns]

if available_metrics:
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, metric in enumerate(available_metrics):
        if i < len(axes):
            ax = axes[i]
            
            # Gráfico de barras
            models_list = list(metrics_df.index)
            values = [metrics_df.loc[model, metric] for model in models_list]
            
            bars = ax.bar(models_list, values, alpha=0.8)
            ax.set_title(f'{metric.upper()} Score', fontsize=14, fontweight='bold')
            ax.set_ylabel('Score', fontsize=12)
            ax.set_ylim(0, 1)
            
            # Adicionar valores nas barras
            for bar, value in zip(bars, values):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
            
            ax.grid(True, alpha=0.3)
    
    # Remover subplots vazios
    for i in range(len(available_metrics), len(axes)):
        fig.delaxes(axes[i])
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Métricas não disponíveis para visualização")

## 5. Análise Qualitativa

In [ ]:
# Análise qualitativa detalhada
def analyze_translation_quality(detailed_results):
    """
    Análise qualitativa das traduções
    """
    print("🔍 ANÁLISE QUALITATIVA DAS TRADUÇÕES")
    print("=" * 60)
    
    # Agrupar por fonte
    by_source = defaultdict(list)
    for result in detailed_results:
        by_source[result['source']].append(result)
    
    for source, translations in by_source.items():
        print(f"\n📝 ORIGINAL: {source}")
        
        # Mostrar referência
        reference = translations[0]['reference']
        print(f"🎯 REFERÊNCIA: {reference}")
        
        print("\n🤖 TRADUÇÕES DOS MODELOS:")
        
        # Ordenar por BLEU-4 score
        translations.sort(key=lambda x: x.get('bleu_4', 0), reverse=True)
        
        for i, trans in enumerate(translations, 1):
            model = trans['model']
            hypothesis = trans['hypothesis']
            bleu4 = trans.get('bleu_4', 0)
            rouge1_f1 = trans.get('rouge1_f1', 0)
            
            print(f"  {i}. {model.upper()}: {hypothesis}")
            print(f"     📊 BLEU-4: {bleu4:.3f} | ROUGE-1 F1: {rouge1_f1:.3f}")
        
        print("-" * 60)

# Executar análise qualitativa
analyze_translation_quality(detailed_results)

## 6. Análise de Erros

In [ ]:
def error_analysis(detailed_results):
    """
    Análise de tipos de erros comuns
    """
    print("🔍 ANÁLISE DE ERROS")
    print("=" * 40)
    
    error_patterns = {
        'word_order': [],
        'missing_words': [],
        'wrong_prepositions': [],
        'grammatical_errors': [],
        'lexical_errors': []
    }
    
    # Análise simplificada de erros
    for result in detailed_results:
        reference = result['reference'].lower()
        hypothesis = result['hypothesis'].lower()
        model = result['model']
        
        # Detectar alguns padrões de erro simples
        ref_words = set(word_tokenize(reference))
        hyp_words = set(word_tokenize(hypothesis))
        
        # Palavras ausentes
        missing = ref_words - hyp_words
        if missing:
            error_patterns['missing_words'].append({
                'model': model,
                'missing': list(missing),
                'sentence': result['source']
            })
        
        # Erros de preposição (exemplos específicos)
        if 'in this' in reference and 'in this' not in hypothesis:
            if 'on this' in hypothesis:
                error_patterns['wrong_prepositions'].append({
                    'model': model,
                    'error': 'in -> on',
                    'sentence': result['source']
                })
    
    # Mostrar resultados
    for error_type, errors in error_patterns.items():
        if errors:
            print(f"\n📋 {error_type.replace('_', ' ').title()}:")
            for error in errors[:3]:  # Mostrar apenas os primeiros 3
                print(f"  • Modelo: {error['model']}")
                if 'missing' in error:
                    print(f"    Palavras ausentes: {error['missing']}")
                elif 'error' in error:
                    print(f"    Erro: {error['error']}")
                print(f"    Frase: {error['sentence']}")
                print()

# Executar análise de erros
error_analysis(detailed_results)

## 7. Comparação de Modelos

In [ ]:
# Ranking dos modelos
def rank_models(metrics_df):
    """
    Cria ranking dos modelos baseado nas métricas
    """
    # Métricas principais para ranking
    key_metrics = ['bleu_4', 'rouge1_f1', 'meteor', 'chrf']
    available_key_metrics = [m for m in key_metrics if m in metrics_df.columns]
    
    if not available_key_metrics:
        print("⚠️ Métricas principais não disponíveis para ranking")
        return
    
    # Calcular score composto (média das métricas normalizadas)
    composite_scores = {}
    
    for model in metrics_df.index:
        scores = []
        for metric in available_key_metrics:
            score = metrics_df.loc[model, metric]
            scores.append(score)
        
        composite_scores[model] = np.mean(scores)
    
    # Ordenar por score composto
    ranked_models = sorted(composite_scores.items(), key=lambda x: x[1], reverse=True)
    
    print("🏆 RANKING DOS MODELOS")
    print("=" * 30)
    
    for i, (model, score) in enumerate(ranked_models, 1):
        medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "📊"
        print(f"{medal} {i}º lugar: {model.upper()}")
        print(f"   Score Composto: {score:.4f}")
        
        # Mostrar métricas individuais
        for metric in available_key_metrics:
            value = metrics_df.loc[model, metric]
            print(f"   {metric}: {value:.4f}")
        print()
    
    return ranked_models

# Executar ranking
ranking = rank_models(metrics_df)

## 8. Salvamento dos Resultados

In [ ]:
# Salvar resultados da avaliação
import os

results_dir = '../docs/evaluation_results'
os.makedirs(results_dir, exist_ok=True)

# Salvar métricas médias
metrics_df.to_csv(f'{results_dir}/average_metrics.csv')

# Salvar resultados detalhados
detailed_df = pd.DataFrame(detailed_results)
detailed_df.to_csv(f'{results_dir}/detailed_results.csv', index=False)

# Salvar ranking
if ranking:
    ranking_df = pd.DataFrame(ranking, columns=['model', 'composite_score'])
    ranking_df.to_csv(f'{results_dir}/model_ranking.csv', index=False)

# Salvar relatório em JSON
evaluation_report = {
    'evaluation_date': datetime.now().isoformat(),
    'test_samples': len(test_df),
    'models_evaluated': list(models.keys()),
    'metrics_used': list(available_metrics) if 'available_metrics' in locals() else [],
    'average_metrics': metrics_df.to_dict(),
    'ranking': ranking if ranking else []
}

with open(f'{results_dir}/evaluation_report.json', 'w', encoding='utf-8') as f:
    json.dump(evaluation_report, f, indent=2, ensure_ascii=False)

print(f"💾 Resultados salvos em: {results_dir}/")
print("📁 Arquivos gerados:")
print("  • average_metrics.csv - Métricas médias por modelo")
print("  • detailed_results.csv - Resultados detalhados")
print("  • model_ranking.csv - Ranking dos modelos")
print("  • evaluation_report.json - Relatório completo")

## 9. Recomendações e Próximos Passos

In [ ]:
# Gerar recomendações baseadas nos resultados
def generate_recommendations(metrics_df, ranking):
    """
    Gera recomendações baseadas na avaliação
    """
    print("💡 RECOMENDAÇÕES E PRÓXIMOS PASSOS")
    print("=" * 50)
    
    if ranking:
        best_model = ranking[0][0]
        best_score = ranking[0][1]
        
        print(f"\n🎯 Melhor Modelo: {best_model.upper()}")
        print(f"   Score Composto: {best_score:.4f}")
        
        if best_score < 0.5:
            print("\n⚠️ ATENÇÃO: Scores baixos detectados")
            print("📋 Recomendações:")
            print("  1. Aumentar dataset de treinamento")
            print("  2. Implementar data augmentation")
            print("  3. Ajustar hiperparâmetros")
            print("  4. Considerar modelos pré-treinados maiores")
        
        elif best_score < 0.7:
            print("\n📈 Performance moderada")
            print("📋 Sugestões de melhoria:")
            print("  1. Fine-tuning mais específico")
            print("  2. Ensemble de modelos")
            print("  3. Pós-processamento das traduções")
        
        else:
            print("\n✅ Boa performance!")
            print("📋 Próximos passos:")
            print("  1. Testes com usuários reais")
            print("  2. Otimização para produção")
            print("  3. Implementação de cache")
    
    print("\n🔧 Melhorias Técnicas Sugeridas:")
    print("  • Implementar beam search para decodificação")
    print("  • Adicionar attention visualization")
    print("  • Implementar back-translation para augmentação")
    print("  • Criar sistema de feedback do usuário")
    
    print("\n📊 Métricas Adicionais para Implementar:")
    print("  • BERTScore para similaridade semântica")
    print("  • Avaliação humana (adequacy & fluency)")
    print("  • Tempo de inferência")
    print("  • Uso de memória")

# Gerar recomendações
generate_recommendations(metrics_df, ranking if 'ranking' in locals() else None)

## 📝 Resumo da Avaliação

### Métricas Implementadas:
- **BLEU (1-4)**: Precisão de n-gramas
- **ROUGE (1, 2, L)**: Recall de n-gramas e sequências longas
- **METEOR**: Alinhamento com sinônimos
- **ChrF**: F-score baseado em caracteres

### Análises Realizadas:
1. **Quantitativa**: Scores automáticos padronizados
2. **Qualitativa**: Comparação manual das traduções
3. **Análise de Erros**: Identificação de padrões problemáticos
4. **Ranking**: Classificação dos modelos por performance

### Próximas Implementações:
1. **Avaliação Humana**: Adequacy e Fluency scores
2. **Métricas Neurais**: BERTScore, BLEURT
3. **Análise de Domínio**: Performance por tipo de texto
4. **Testes A/B**: Comparação com usuários reais

---

**Desenvolvido para o NeuroTranslator PT-EN** 🧠🔄🌐